<a href="https://colab.research.google.com/github/hellocybernetics/TensorFlow_Eager_Execution_Tutorials/blob/master/tutorials/03_advanced/graph_convnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nightly-2.0-preview

In [0]:
import math
import tensorflow as tf

In [11]:
class GraphConvolution(tf.keras.layers.Layer):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, out_features, use_bias=True):
        super(GraphConvolution, self).__init__()
        
        self.out_features = out_features        
        self.use_bias = use_bias
    
    def build(self, input_shape):
        
        stddv = 1 / math.sqrt(self.out_features)
        initializer = tf.keras.initializers.RandomUniform(
            minval=-stddv,
            maxval=stddv
        )
        
        self.kernel = self.add_variable(
            name="kernel", 
            shape=[int(input_shape[-1]), self.out_features],
            initializer=initializer
        )
        
        if self.use_bias:
            self.bias = self.add_variable(
                name="bias",
                shape=[self.out_features],
                initializer=initializer
            )
    def call(self, input, adj):
        support = tf.matmul(input, self.kernel)
        output = tf.sparse.sparse_dense_matmul(adj, support)
        if self.bias:
            return output + self.bias
        else:
            return output

    def __repr__(self):

    
        return self.__class__.__name__ + ' (' \
               + "output features is : " \
               + str(self.out_features) + ')'

layer = GraphConvolution(10)
layer

GraphConvolution (output features is : 10)

In [0]:
layer = GraphConvolution(5)

In [0]:
class GCN(tf.keras.Model):
    def __init__(self, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nhid)
        self.gc2 = GraphConvolution(nclass)
        self.dropout = tf.keras.layers.Dropout(dropout)

    def forward(self, x, adj, training=False):
        x = tf.relu(self.gc1(x, adj))
        x = self.dropout(x, self.dropout, training=training)
        x = self.gc2(x, adj)
        return tf.nn.log_softmax(x, axis=1)

In [0]:
model = GCN(50, 2, 0.3)

In [15]:
model